In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
import numpy as np

# Make team names uniform between sheets

In [2]:
names_file ='/Users/stefanfeiler/Desktop/for_micah/names.csv'

In [3]:
df = pd.read_csv(names_file)

In [4]:
mapping_dict = {key: value for key, value in zip(df['abbrev'], df['full_name'])}

In [5]:
extra_data_file = '/Users/stefanfeiler/Desktop/for_micah/2023_mlb_extra.csv'

In [6]:
extra_df = pd.read_csv(extra_data_file)

In [7]:
extra_df = extra_df.replace(mapping_dict)

In [8]:
extra_data_file_new = '/Users/stefanfeiler/Desktop/for_micah/mlb_extra_stats_uniform_names.csv'

In [9]:
extra_df.to_csv(extra_data_file_new)

# Step 2: Convert all times to MST

In [10]:
raw_part_1 = pd.read_csv('/Users/stefanfeiler/Desktop/data/mlb_raw_2023_07_PART1.csv')
raw_part_2 = pd.read_csv('/Users/stefanfeiler/Desktop/data/mlb_raw_2023_07_PART2.csv')
raw = pd.concat([raw_part_1, raw_part_2])

/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_31287/2425686787.py:1: DtypeWarning: Columns (158,160) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_part_1 = pd.read_csv('/Users/stefanfeiler/Desktop/data/mlb_raw_2023_07_PART1.csv')
/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_31287/2425686787.py:2: DtypeWarning: Columns (158,160) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_part_2 = pd.read_csv('/Users/stefanfeiler/Desktop/data/mlb_raw_2023_07_PART2.csv')


In [11]:
raw.head(10).to_csv('/Users/stefanfeiler/Desktop/data/mlb_raw_2023_07_columns.csv')

In [12]:
def find_different_elements(list1, list2):
    # Convert lists to sets
    set1 = set(list1)
    set2 = set(list2)

    # Find elements that are in set1 but not in set2
    unique_to_set1 = set1 - set2

    # Find elements that are in set2 but not in set1
    unique_to_set2 = set2 - set1

    return list(unique_to_set1), list(unique_to_set2)

# Example lists
list1 = [1, 2, 3, 4, 5]
list2 = [3, 4, 5, 6, 7]

# Find different elements
diff_list1, diff_list2 = find_different_elements(raw_part_2.columns, raw_part_1.columns)

print("Elements unique to list1:", diff_list1)
print("Elements unique to list2:", diff_list2)

Elements unique to list1: []
Elements unique to list2: []


In [13]:
raw.iloc[0]

game_id                  7f989faf14324e706845194a889bf62d
commence_time                               7/19/23 16:35
time_pulled                                 7/19/23 15:55
home_team                             Cleveland Guardians
away_team                              Pittsburgh Pirates
                                       ...               
williamhill_us_2_time                       7/19/23 15:50
wynnbet_1_odds                                       1.75
wynnbet_1_time                              7/19/23 15:48
wynnbet_2_odds                                       2.21
wynnbet_2_time                              7/19/23 15:48
Name: 0, Length: 161, dtype: object

In [76]:
# # This takes 2.5 minutes
time_columns = [col for col in raw.columns if 'time' in col]
raw[time_columns] = raw[time_columns].apply(lambda x: pd.to_datetime(x, format="%m/%d/%y %H:%M"))

In [77]:
#raw['commence_time'] = raw['commence_time'] - pd.Timedelta(hours=7)
raw[time_columns] = raw[time_columns]- pd.Timedelta(hours=7)

In [78]:
raw.head()

,game_id,commence_time,time_pulled,home_team,away_team,barstool_1_odds,barstool_1_time,barstool_2_odds,barstool_2_time,betclic_1_odds,...,williamhill_2_odds,williamhill_2_time,williamhill_us_1_odds,williamhill_us_1_time,williamhill_us_2_odds,williamhill_us_2_time,wynnbet_1_odds,wynnbet_1_time,wynnbet_2_odds,wynnbet_2_time
0,7f989faf14324e706845194a889bf62d,2023-07-19 09:35:00,2023-07-19 08:55:00,Cleveland Guardians,Pittsburgh Pirates,1.71,2023-07-19 08:50:00,2.20,2023-07-19 08:50:00,1.72,...,2.20,2023-07-19 08:48:00,1.67,2023-07-19 08:50:00,2.26,2023-07-19 08:50:00,1.75,2023-07-19 08:48:00,2.21,2023-07-19 08:48:00
1,896fe645cdb1c1d6b96e8b766ee1aa8d,2023-07-19 10:05:00,2023-07-19 08:55:00,Baltimore Orioles,Los Angeles Dodgers,2.25,2023-07-19 08:50:00,1.69,2023-07-19 08:50:00,2.25,...,1.62,2023-07-19 08:49:00,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2,eb029fd843bb81f3a097861cfedd182f,2023-07-19 11:05:00,2023-07-19 08:55:00,Tampa Bay Rays,Texas Rangers,2.00,2023-07-19 08:50:00,1.83,2023-07-19 08:50:00,2.00,...,1.82,2023-07-19 08:49:00,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
3,006986994ed204bf0e24253f1fbff107,2023-07-19 11:15:00,2023-07-19 08:55:00,Miami Marlins,St. Louis Cardinals,1.91,2023-07-19 08:50:00,1.91,2023-07-19 08:50:00,1.91,...,1.90,2023-07-19 08:49:00,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
4,99a5d553a4d308dd1c335ac53322084a,2023-07-19 12:10:00,2023-07-19 08:55:00,Colorado Rockies,Houston Astros,2.30,2023-07-19 08:50:00,1.67,2023-07-19 08:50:00,2.27,...,1.65,2023-07-19 08:50:00,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT


# Step 3: Make the same my_game_id values in each sheet

In [79]:
extra = pd.read_csv(extra_data_file_new)

In [80]:
extra['home_team_final'] = extra[['home_team', 'away_team']].min(axis=1)
extra['away_team_final'] = extra[['home_team', 'away_team']].max(axis=1)

In [81]:
raw['date'] = raw['commence_time'].copy().dt.strftime('%Y%m%d')
extra['date']= extra['date'].astype(str)

In [82]:
raw['commence_time']

0        2023-07-19 09:35:00
1        2023-07-19 10:05:00
2        2023-07-19 11:05:00
3        2023-07-19 11:15:00
4        2023-07-19 12:10:00
                 ...        
282263   2023-03-30 13:10:00
282264   2023-03-30 16:08:00
282265   2023-03-30 19:07:00
282266   2023-03-30 19:10:00
282267   2023-03-30 19:10:00
Name: commence_time, Length: 635183, dtype: datetime64[ns]

In [83]:
raw['my_id'] = raw['home_team'] + raw['away_team'] + raw['date']
extra['my_id'] = extra['home_team_final'] + extra['away_team_final'] + extra['date']

# Step 4: Make a complicated dict

In [84]:
# Make a dict that maps raw my_id to raw game_id and includes duplicates
mapping_dict = {}
for key, value in zip(raw['my_id'], raw['game_id']):
    if key in mapping_dict:
        mapping_dict[key].append(value)  # Append value to existing key's list
        mapping_dict[key] = list(set(mapping_dict[key]))
    else:
        mapping_dict[key] = [value]  

In [85]:
count = 0
for key, values in mapping_dict.items():
    if len(values) == 2:
        count+=1
print(count)

21


In [86]:
print(len(mapping_dict))

1459


In [87]:
extra['number_of_game_today'] = extra['number_of_game_today'].astype(str)

In [88]:
# Make a dict that maps extra my_id to the extra game_value (double header or not)
extra_dict = {}
count = 0
for key, value in zip(extra['my_id'], extra['number_of_game_today']):
    if key in extra_dict:
        extra_dict[key].append(value)
        extra_dict[key] = list(extra_dict[key])
    else:
        extra_dict[key] = [value]
print(len(extra_dict))

2424


In [89]:
# Make a dict that maps game_id's to commence_times
id_commence_dict = {key: value for key, value in zip(raw['game_id'], raw['commence_time'])}

In [90]:
# Go through each mapping_dict item
# if the length of values is 1, put it in the new dict with the suffix '_0'
# if the length of values is 2, put it in the new dict with the proper suffix
new_dict = {}
for key, values in mapping_dict.items():
    if len(values) == 1:
        new_dict[str(str(key) + '_0')] = values[0]
    elif len(values) == 2:
        if id_commence_dict[values[0]] < id_commence_dict[values[1]]:
            new_dict[str(str(key) + '_1')] = values[0]
            new_dict[str(str(key) + '_2')] = values[1]
        elif id_commence_dict[values[0]] > id_commence_dict[values[1]]:
            new_dict[str(str(key) + '_2')] = values[0]
            new_dict[str(str(key) + '_1')] = values[1]

In [91]:
extra['my_id_game'] = extra['home_team_final'] + extra['away_team_final'] + extra['date'] + '_' + extra['number_of_game_today']

In [92]:
my_id_game = extra['my_id_game'].tolist()

In [93]:
count = 0
new_list = []
for key, value in new_dict.items():
    if key in my_id_game:
        new_list.append(key)


In [94]:
count = 0
for item in new_list:
    if item in my_id_game:
        count +=1
print(count)

1426


In [95]:
# Takes about 2 minutes
raw['my_game_id_final'] = raw['game_id'].apply(lambda x: next((k for k, v in new_dict.items() if v == x), None))
raw['my_game_id_final']

0           Cleveland GuardiansPittsburgh Pirates20230719_0
1            Baltimore OriolesLos Angeles Dodgers20230719_0
2                     Tampa Bay RaysTexas Rangers20230719_0
3                Miami MarlinsSt. Louis Cardinals20230719_0
4                  Colorado RockiesHouston Astros20230719_0
                                ...                        
282263       St. Louis CardinalsToronto Blue Jays20230330_0
282264            Chicago White SoxHouston Astros20230330_0
282265        Los Angeles AngelsOakland Athletics20230330_0
282266    Arizona DiamondbacksLos Angeles Dodgers20230330_0
282267        Cleveland GuardiansSeattle Mariners20230330_0
Name: my_game_id_final, Length: 635183, dtype: object

In [96]:
#raw['winning_team'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['winning_team'])

raw['number_of_game_today'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['number_of_game_today'])

raw['day_of_week'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['day_of_week'])

raw['away_team'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['away_team'])
raw['away_team_league'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['away_team_league'])
raw['away_team_game_number'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['away_team_game_number'])

raw['home_team'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['home_team'])
raw['home_team_league'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['home_team_league'])
raw['home_team_game_number'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['home_team_game_number'])

raw['day_night'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['day_night'])
raw['park_id'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['park_id'])

In [97]:
raw.number_of_game_today.value_counts()

0    621242
Name: number_of_game_today, dtype: int64

In [34]:
raw_first_1000 = raw.head(1000)
raw_first_1000.to_csv('/Users/stefanfeiler/Desktop/for_micah/outData_mlb_raw_final_first_1000.csv',index=False)

In [98]:
import regex as re

In [99]:
from datetime import datetime

def convert_date_string_to_digits(date_str):
    # Define the input and output date formats
    input_format = "%A, %B %d, %Y"
    output_format = "%Y%m%d"

    # Convert the input date string to a datetime object
    date_obj = datetime.strptime(date_str, input_format)

    # Convert the datetime object back to the desired string format
    return date_obj.strftime(output_format)

# Example usage:
date_string = "Thursday, March 30, 2023"
result = convert_date_string_to_digits(date_string)

In [100]:
test = "Colorado Rockies (7) @ San Diego Padres (2)   Boxscore"
out = re.match(r'.+\(([0-9]{1,2})\).+\(([0-9]{1,2})\)', test)
print(out)
out[2]

<regex.Match object; span=(0, 43), match='Colorado Rockies (7) @ San Diego Padres (2)'>


'2'

In [101]:
with open('/Users/stefanfeiler/Desktop/for_micah/this_season_scores.txt','r', encoding = 'utf-16') as f:
    lines = f.readlines()

df = pd.DataFrame(columns=['date','away_team','away_score','home_team','home_score','game_id','winner_home'])
j = 0
date=0
good=0
length = len(lines)
counter = 0
for i in lines:
    if re.match(r'\s', i):
        j+=1
    elif i.startswith("\""):
        date+=1
        date_string = i.strip().strip("\"")
        date_num = convert_date_string_to_digits(date_string)

    else:
        good+=1
        out = re.match(r'(.+) \(([0-9]{1,2})\) @ (.+) \(([0-9]{1,2})\)', i)
        away_team = str(out[1])
        home_team = str(out[3])
        away_score = int(out[2])
        home_score = int(out[4])
        if away_score > home_score:
            home_winner = False
        elif home_score > away_score:
            home_winner = True
        else: "shout and scream"
        game_id = away_team + home_team + str(date_num)
        game_data = {
            'date': date_num,
            'away_team': away_team,
            'away_score': away_score,
            'home_team': home_team,
            'home_score': home_score,
            'game_id': game_id,  # You might need to define 'counter' somewhere.
            'winner_home': home_winner
        }
        df.loc[len(df.index)] = game_data 


#innerjoin by game_id

In [102]:
raw.head()

,game_id,commence_time,time_pulled,home_team,away_team,barstool_1_odds,barstool_1_time,barstool_2_odds,barstool_2_time,betclic_1_odds,...,my_id,my_game_id_final,number_of_game_today,day_of_week,away_team_league,away_team_game_number,home_team_league,home_team_game_number,day_night,park_id
0,7f989faf14324e706845194a889bf62d,2023-07-19 09:35:00,2023-07-19 08:55:00,Pittsburgh Pirates,Cleveland Guardians,1.71,2023-07-19 08:50:00,2.20,2023-07-19 08:50:00,1.72,...,Cleveland GuardiansPittsburgh Pirates20230719,Cleveland GuardiansPittsburgh Pirates20230719_0,0,Wed,AL,96.0,NL,96.0,N,PIT08
1,896fe645cdb1c1d6b96e8b766ee1aa8d,2023-07-19 10:05:00,2023-07-19 08:55:00,Baltimore Orioles,Los Angeles Dodgers,2.25,2023-07-19 08:50:00,1.69,2023-07-19 08:50:00,2.25,...,Baltimore OriolesLos Angeles Dodgers20230719,Baltimore OriolesLos Angeles Dodgers20230719_0,0,Wed,NL,95.0,AL,95.0,D,WIL02
2,eb029fd843bb81f3a097861cfedd182f,2023-07-19 11:05:00,2023-07-19 08:55:00,Texas Rangers,Tampa Bay Rays,2.00,2023-07-19 08:50:00,1.83,2023-07-19 08:50:00,2.00,...,Tampa Bay RaysTexas Rangers20230719,Tampa Bay RaysTexas Rangers20230719_0,0,Wed,AL,99.0,AL,97.0,D,ARL03
3,006986994ed204bf0e24253f1fbff107,2023-07-19 11:15:00,2023-07-19 08:55:00,St. Louis Cardinals,Miami Marlins,1.91,2023-07-19 08:50:00,1.91,2023-07-19 08:50:00,1.91,...,Miami MarlinsSt. Louis Cardinals20230719,Miami MarlinsSt. Louis Cardinals20230719_0,0,Wed,NL,98.0,NL,96.0,D,STL10
4,99a5d553a4d308dd1c335ac53322084a,2023-07-19 12:10:00,2023-07-19 08:55:00,Colorado Rockies,Houston Astros,2.30,2023-07-19 08:50:00,1.67,2023-07-19 08:50:00,2.27,...,Colorado RockiesHouston Astros20230719,Colorado RockiesHouston Astros20230719_0,0,Wed,AL,96.0,NL,96.0,D,DEN02


In [103]:
#innerjoin by game_id in df equals  my_id in raw_last_1000
merged_df = pd.merge(df, raw, left_on='game_id', right_on='my_id', how='inner')
merged_df.head()
merged_df.number_of_game_today.value_counts()

0    270289
Name: number_of_game_today, dtype: int64

In [104]:
merged_df.head()

,date_x,away_team_x,away_score,home_team_x,home_score,game_id_x,winner_home,game_id_y,commence_time,time_pulled,...,my_id,my_game_id_final,number_of_game_today,day_of_week,away_team_league,away_team_game_number,home_team_league,home_team_game_number,day_night,park_id
0,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:35:00,...,Baltimore OriolesBoston Red Sox20230330,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07
1,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:30:00,...,Baltimore OriolesBoston Red Sox20230330,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07
2,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:25:00,...,Baltimore OriolesBoston Red Sox20230330,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07
3,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:20:00,...,Baltimore OriolesBoston Red Sox20230330,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07
4,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:15:00,...,Baltimore OriolesBoston Red Sox20230330,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07


In [105]:
merged_df.rename(columns={'away_team_x' : 'team_1'}, inplace=True)
merged_df.rename(columns={'home_team_x' : 'team_2'}, inplace=True)
merged_df.rename(columns={'away_team_y' : 'away_team'}, inplace=True)
merged_df.rename(columns={'home_team_y' : 'home_team'}, inplace=True)
merged_df.rename(columns={'game_id_y' : 'game_id'}, inplace=True)
merged_df.rename(columns={'date_y' : 'date'}, inplace=True)

In [106]:
merged_df['winning_team'] = np.where(merged_df['winner_home'] == True, merged_df['home_team'], merged_df['away_team'])

/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_3579/226154304.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df['winning_team'] = np.where(merged_df['winner_home'] == True, merged_df['home_team'], merged_df['away_team'])


In [110]:
# Observations where we don't have winning team data. If we update our score data from the same site these should decrease
nan_count = merged_df['winning_team'].isna().sum()

print("Number of NaN values in 'winning_team':", nan_count)

# Drop rows where the value of 'Column_Name' is NaN
merged_df = merged_df.dropna(subset=['winning_team'], axis=0)

nan_count = merged_df['winning_team'].isna().sum()

print("Number of NaN values in 'winning_team' after drop:", nan_count)

Number of NaN values in 'winning_team': 12083
Number of NaN values in 'winning_team' after drop: 0


In [111]:
merged_df.head()

,date_x,team_1,away_score,team_2,home_score,game_id_x,winner_home,game_id,commence_time,time_pulled,...,my_game_id_final,number_of_game_today,day_of_week,away_team_league,away_team_game_number,home_team_league,home_team_game_number,day_night,park_id,winning_team
0,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:35:00,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles
1,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:30:00,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles
2,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:25:00,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles
3,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:20:00,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles
4,20230330,Baltimore Orioles,10,Boston Red Sox,9,Baltimore OriolesBoston Red Sox20230330,False,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:15:00,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles


In [125]:
merged_df.columns.tolist()

['team_1',
 'team_2',
 'game_id',
 'commence_time',
 'time_pulled',
 'home_team',
 'away_team',
 'barstool_1_odds',
 'barstool_1_time',
 'barstool_2_odds',
 'barstool_2_time',
 'betclic_1_odds',
 'betclic_1_time',
 'betclic_2_odds',
 'betclic_2_time',
 'betfair_1_odds',
 'betfair_1_time',
 'betfair_2_odds',
 'betfair_2_time',
 'betfred_1_odds',
 'betfred_1_time',
 'betfred_2_odds',
 'betfred_2_time',
 'betmgm_1_odds',
 'betmgm_1_time',
 'betmgm_2_odds',
 'betmgm_2_time',
 'betonlineag_1_odds',
 'betonlineag_1_time',
 'betonlineag_2_odds',
 'betonlineag_2_time',
 'betrivers_1_odds',
 'betrivers_1_time',
 'betrivers_2_odds',
 'betrivers_2_time',
 'betus_1_odds',
 'betus_1_time',
 'betus_2_odds',
 'betus_2_time',
 'betway_1_odds',
 'betway_1_time',
 'betway_2_odds',
 'betway_2_time',
 'bovada_1_odds',
 'bovada_1_time',
 'bovada_2_odds',
 'bovada_2_time',
 'casumo_1_odds',
 'casumo_1_time',
 'casumo_2_odds',
 'casumo_2_time',
 'circasports_1_odds',
 'circasports_1_time',
 'circasports_2_od

In [50]:
# Did this part manually in excel before prepreocessing

# cols_to_drop = ['date_x', 'away_score', 'home_score', 'game_id_x', 'winner_home', '1.7','7/19/23 15:50','2.2','7/19/23 15:50.1','1.73', '7/19/23 15:49','2.15','7/19/23 15:49.1','1.73.1','7/19/23 15:50.2','2.15.1','7/19/23 15:50.3','1.75','7/19/23 15:50.4','2.3','7/19/23 15:50.5','1.75.1','7/19/23 15:49.2','2.3.1','7/19/23 15:49.3','1.65','7/19/23 15:49.4','2.15.2','7/19/23 15:49.5']

# merged_df.drop(columns=cols_to_drop, inplace=True)

In [114]:
cols_to_drop = ['date_x', 'away_score', 'home_score', 'game_id_x', 'winner_home']
merged_df.drop(columns=cols_to_drop, inplace=True)

In [115]:
len(merged_df.columns.tolist())

175

In [120]:
merged_df.to_parquet("/Users/stefanfeiler/Desktop/SmartBetter_site/website/mlb_data/2023_data_for_val.parquet")

In [123]:
merged_df.to_csv("/Users/stefanfeiler/Desktop/SmartBetter_site/website/mlb_data/2023_data_for_val.csv")

In [122]:
len(merged_df)

270289

In [121]:
merged_df.head()

,team_1,team_2,game_id,commence_time,time_pulled,home_team,away_team,barstool_1_odds,barstool_1_time,barstool_2_odds,...,my_game_id_final,number_of_game_today,day_of_week,away_team_league,away_team_game_number,home_team_league,home_team_game_number,day_night,park_id,winning_team
0,Baltimore Orioles,Boston Red Sox,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:35:00,Boston Red Sox,Baltimore Orioles,1.04,2023-03-30 14:25:00,10.0,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles
1,Baltimore Orioles,Boston Red Sox,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:30:00,Boston Red Sox,Baltimore Orioles,1.04,2023-03-30 14:25:00,10.0,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles
2,Baltimore Orioles,Boston Red Sox,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:25:00,Boston Red Sox,Baltimore Orioles,1.04,2023-03-30 14:25:00,10.0,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles
3,Baltimore Orioles,Boston Red Sox,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:20:00,Boston Red Sox,Baltimore Orioles,1.16,2023-03-30 14:19:00,5.1,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles
4,Baltimore Orioles,Boston Red Sox,f2912049aeca157ded4e77586ad96b6f,2023-03-30 11:10:00,2023-03-30 14:15:00,Boston Red Sox,Baltimore Orioles,1.16,2023-03-30 14:15:00,5.1,...,Baltimore OriolesBoston Red Sox20230330_0,0,Thu,AL,1.0,AL,1.0,D,BOS07,Baltimore Orioles


THIS IS WHERE MICAHS WORK STOPS. THIS IS ONLY A SUBSET OF THE DATA BUT CAN BE EASILY ADAPTED ONCE WE HAVE ALL DATA

In [27]:
raw_first_1000 = raw.head(1000)
raw_first_1000.to_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_first_1000.csv',index=False)

In [28]:
raw.to_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.csv',index=False)

In [29]:
raw.to_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet', index=False)

In [30]:
raw = pd.read_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet')

# DONE!!!!

In [31]:
# Some extra cleaning. We want to drop all columns where we don't know 

# BEFORE WE DO ANYTHING, WE NEED TO READ IN THE FULL RAW CSV AND SAVE THE TEAM_1 AND TEAM_2 COLUMNS BACK INTO THE FINAL DF

raw = pd.read_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet')

In [32]:
raw_with_alpha_teams = pd.read_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_full.csv')

/Users/stefanfeiler/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (90,92,94,96,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
raw_home_away = pd.read_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet')

In [34]:
print(len(raw_with_alpha_teams))

921714


In [35]:
print(len(raw_home_away))

921714


In [36]:
raw_home_away['team_1'] = raw_with_alpha_teams['home_team']
raw_home_away['team_2'] = raw_with_alpha_teams['away_team']

In [37]:
raw_home_away.to_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet', index=False)

# More recent data

In [1]:
# just found some more recent data. Have to process 2023 season up through the current date rn....

In [6]:
names_file ='/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/names.csv'

In [7]:
df = pd.read_csv(names_file)

In [8]:
mapping_dict = {key: value for key, value in zip(df['abbrev'], df['full_name'])}

In [9]:
extra_data_file = '/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/2023SKED.csv'

In [10]:
extra_df = pd.read_csv(extra_data_file)

In [11]:
extra_df = extra_df.replace(mapping_dict)

In [18]:
extra_df.to_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/2023SKED_uniform_names.csv')

In [19]:
# Nevermind! We don't have the scores.....

In [ ]:
# Nevermind again! Suck my cock 

In [14]:
import pandas as pd
import plotly.express as px

In [15]:
df = pd.read_csv('/Users/stefanfeiler/Desktop/another test final tonight copy.csv')

In [16]:
# Step 1: Prepare the data
df_counts = df.groupby(['team', 'target']).size().reset_index(name='count')

# Step 2: Create the bar graph using Plotly
fig = px.bar(df_counts, x='team', y='count', color='target',
             labels={'team': 'Team', 'count': 'Count'},
             title='Distribution of Categories in DataFrame Colored by Binary Column')

# If you want to adjust the appearance of the graph, you can use the following optional parameters:
# fig.update_traces(marker_line_width=1.5, opacity=0.6)

# Show the plot
fig.show()

In [29]:
df1 = pd.read_csv('/Users/stefanfeiler/Desktop/scores_test.csv')
df2 = pd.read_csv('/Users/stefanfeiler/Desktop/SmartBetter high risk copy.csv')

In [45]:
game_winners = df1.set_index('game_id')['winning_team'].to_dict()

In [57]:
def fill_na_with_winner(row):
    game_id = row['game_id']
    team = row['team']
    opponenet = row['opponent']
    winning_team = game_winners.get(game_id)
    if team == winning_team:
        return 1
    elif opponenet == winning_team:
        return 0
    else:
        return row['target']

# Apply the function to fill NA values
df2['target'] = df2.apply(lambda row: fill_na_with_winner(row), axis=1)

In [58]:
df2.to_csv('/Users/stefanfeiler/Desktop/scores_filled.csv')

In [36]:
df_merged = pd.merge(df1, df2, on='game_id', how='inner')

In [39]:
df_merged['target'] = (df_merged['winning_team'] == df_merged['team']).astype(int)

In [40]:
df_merged.to_csv('/Users/stefanfeiler/Desktop/merge_test.csv')

In [42]:
game_id_target_map = df_merged.set_index('game_id')['target'].to_dict()

In [43]:
game_id_target_map

{'96d47d41251c8e6991fe067926d156bf': 1,
 '928d216ad34dc0eea0c54425d8bd5eea': 0,
 'd84e8a2218b10737e16c22820555d118': 0,
 'f6a9c25fe8e1d7cc9cafbf6cebeaa890': 0,
 '8ab5e3b027cad19aef74c4886941ad1b': 0}

In [126]:
df = pd.read_csv('/Users/stefanfeiler/Desktop/SmartBetter_site/website/users/user_strategy_names.csv')
text_allowed = df[df['text_alerts'] == True ]

In [127]:
text_allowed

,username,strategy_name,text_alerts
0,youngfeiler,SmartBetter low risk,True
3,youngfeiler,big mama,True
9,chrisfeiler,big mama,True


In [433]:
phone_nums = pd.read_csv('/Users/stefanfeiler/Desktop/SmartBetter_site/website/users/login_info.csv')

In [434]:
merged_df = phone_nums.merge(text_allowed, on='username', how='inner')

In [435]:
merged_df_info = merged_df[['phone', 'strategy_name']]

In [436]:
merged_df_info

,phone,strategy_name
0,13035796207,SmartBetter low risk
1,13035796207,big mama
2,13037176950,big mama


In [437]:
grouped_dict = merged_df_info.groupby('phone')['strategy_name'].agg(list).to_dict()
grouped_dict

{13035796207: ['SmartBetter low risk', 'big mama'], 13037176950: ['big mama']}

In [438]:
from twilio.rest import Client
account_sid = 'AC8c14921fcae1fead46c9a56bbd46d606'
auth_token = 'a3bfa3894e91404fe1d53c6f2fb8246a'
twilio_phone_number = '+18573922435'
client = Client(account_sid, auth_token)

In [442]:
for phone_number, strategy_list  in grouped_dict.items():
  strategies_for_user = ''.join([f'\n"{element}"' for element in strategy_list])
  message_body = f'Smart Alert:\n\nNew info available for one of your strategies \n\nAct now:\n'
  print(message_body)
  print('-------')
  message = client.messages.create(
                  body=message_body,
                  from_=twilio_phone_number,
                  to=phone_number
              )

Smart Alert:

New info available for one of your strategies 

Act now:

-------
Smart Alert:

New info available for one of your strategies 

Act now:

-------


In [304]:
df = pd.read_csv('/Users/stefanfeiler/Desktop/test_live_stacked 6.36.22 PM.csv')

In [417]:
time_cols = [col for col in df.columns if '_1_time' in col]
odds_cols = [col for col in df.columns if '_1_odds' in col]
odds_df = df[odds_cols]
snap_time_col = pd.DataFrame()
snap_time_col = pd.to_timedelta(df['snapshot_time'].astype(str))
subset_columns = [col for col in odds_cols if any(item in col for item in ['barstool', 'betmgm', 'draftkings', 'bovada', 'coral', 'fanduel'])]

# df['snapshot_time'] = pd.to_timedelta(df['snapshot_time'].astype(str))
time_df = pd.DataFrame()

for col in time_cols:
    time_df[col] = pd.to_timedelta(pd.to_datetime(df[col]).dt.time.astype(str))

result_df = time_df.sub(snap_time_col, axis=0)

In [418]:
subset_columns

['barstool_1_odds',
 'betmgm_1_odds',
 'bovada_1_odds',
 'coral_1_odds',
 'draftkings_1_odds',
 'fanduel_1_odds']

In [419]:
result_df = result_df.abs()

In [420]:
result_df.head()

,barstool_1_time,betclic_1_time,betfair_1_time,betfred_1_time,betmgm_1_time,betonlineag_1_time,betrivers_1_time,betus_1_time,betway_1_time,bovada_1_time,...,sugarhouse_1_time,superbook_1_time,twinspires_1_time,unibet_1_time,unibet_eu_1_time,unibet_uk_1_time,unibet_us_1_time,williamhill_1_time,williamhill_us_1_time,wynnbet_1_time
0,0 days 00:00:39,0 days 00:00:08,0 days 20:56:56,0 days 20:56:56,0 days,0 days 20:56:56,0 days 20:56:56,0 days 20:56:56,0 days 20:56:56,0 days 00:00:38,...,0 days 20:56:56,0 days 20:56:56,0 days 00:00:25,0 days 20:56:56,0 days 00:00:25,0 days 00:00:06,0 days,0 days 20:56:56,0 days 20:56:56,0 days 20:56:56
1,0 days 00:00:39,0 days 00:00:08,0 days 20:56:56,0 days 20:56:56,0 days,0 days 20:56:56,0 days 20:56:56,0 days 20:56:56,0 days 20:56:56,0 days 00:00:38,...,0 days 20:56:56,0 days 20:56:56,0 days 00:00:25,0 days 20:56:56,0 days 00:00:25,0 days 00:00:06,0 days,0 days 20:56:56,0 days 20:56:56,0 days 20:56:56
2,0 days 00:00:39,0 days 00:00:08,0 days 20:56:56,0 days 20:56:56,0 days,0 days 20:56:56,0 days 20:56:56,0 days 20:56:56,0 days 20:56:56,0 days 00:00:38,...,0 days 20:56:56,0 days 20:56:56,0 days 00:00:25,0 days 20:56:56,0 days 00:00:25,0 days 00:00:06,0 days,0 days 20:56:56,0 days 20:56:56,0 days 20:56:56
3,0 days 00:00:39,0 days 00:00:08,0 days 20:56:56,0 days 20:56:56,0 days,0 days 00:00:25,0 days 00:00:25,0 days 00:00:33,0 days 20:56:56,0 days 00:00:38,...,0 days 20:56:56,0 days 00:00:34,0 days 00:00:25,0 days 20:56:56,0 days 00:00:25,0 days 00:00:06,0 days,0 days 00:00:45,0 days 00:00:31,0 days 00:03:12
4,0 days 00:00:39,0 days 00:00:08,0 days 20:56:56,0 days 20:56:56,0 days,0 days 00:00:25,0 days 00:00:25,0 days 00:00:33,0 days 20:56:56,0 days 00:00:38,...,0 days 20:56:56,0 days 00:00:34,0 days 00:00:25,0 days 20:56:56,0 days 00:00:25,0 days 00:00:06,0 days,0 days 00:00:45,0 days 00:00:31,0 days 00:03:12


In [421]:
odds_df.head()

,barstool_1_odds,betclic_1_odds,betfair_1_odds,betfred_1_odds,betmgm_1_odds,betonlineag_1_odds,betrivers_1_odds,betus_1_odds,betway_1_odds,bovada_1_odds,...,sugarhouse_1_odds,superbook_1_odds,twinspires_1_odds,unibet_1_odds,unibet_eu_1_odds,unibet_uk_1_odds,unibet_us_1_odds,williamhill_1_odds,williamhill_us_1_odds,wynnbet_1_odds
0,1.10,1.10,0.0,0.0,1.13,0.00,0.00,0.00,0.0,1.09,...,0.0,0.00,1.13,0.0,1.13,1.12,1.14,0.00,0.00,0.00
1,1.05,1.06,0.0,0.0,1.05,0.00,0.00,0.00,0.0,1.05,...,0.0,0.00,1.01,0.0,1.01,1.04,1.04,0.00,0.00,0.00
2,1.83,1.85,0.0,0.0,1.87,0.00,0.00,0.00,0.0,1.77,...,0.0,0.00,1.82,0.0,1.82,1.89,1.85,0.00,0.00,0.00
3,2.10,2.15,0.0,0.0,2.10,2.14,2.16,2.16,0.0,2.08,...,0.0,2.15,2.15,0.0,2.16,2.15,2.16,2.10,2.10,2.15
4,1.77,1.78,0.0,0.0,1.77,1.79,1.74,1.77,0.0,1.76,...,0.0,1.78,1.74,0.0,1.74,1.74,1.74,1.77,1.77,1.80


In [422]:
threshold = pd.Timedelta(seconds=30)

In [423]:
mask = result_df <= threshold
mask.head()

,barstool_1_time,betclic_1_time,betfair_1_time,betfred_1_time,betmgm_1_time,betonlineag_1_time,betrivers_1_time,betus_1_time,betway_1_time,bovada_1_time,...,sugarhouse_1_time,superbook_1_time,twinspires_1_time,unibet_1_time,unibet_eu_1_time,unibet_uk_1_time,unibet_us_1_time,williamhill_1_time,williamhill_us_1_time,wynnbet_1_time
0,False,True,False,False,True,False,False,False,False,False,...,False,False,True,False,True,True,True,False,False,False
1,False,True,False,False,True,False,False,False,False,False,...,False,False,True,False,True,True,True,False,False,False
2,False,True,False,False,True,False,False,False,False,False,...,False,False,True,False,True,True,True,False,False,False
3,False,True,False,False,True,True,True,False,False,False,...,False,False,True,False,True,True,True,False,False,False
4,False,True,False,False,True,True,True,False,False,False,...,False,False,True,False,True,True,True,False,False,False


In [424]:
# Create a new mask DataFrame with the correct column names
mask.columns = odds_df.columns


In [425]:
mask.head()

,barstool_1_odds,betclic_1_odds,betfair_1_odds,betfred_1_odds,betmgm_1_odds,betonlineag_1_odds,betrivers_1_odds,betus_1_odds,betway_1_odds,bovada_1_odds,...,sugarhouse_1_odds,superbook_1_odds,twinspires_1_odds,unibet_1_odds,unibet_eu_1_odds,unibet_uk_1_odds,unibet_us_1_odds,williamhill_1_odds,williamhill_us_1_odds,wynnbet_1_odds
0,False,True,False,False,True,False,False,False,False,False,...,False,False,True,False,True,True,True,False,False,False
1,False,True,False,False,True,False,False,False,False,False,...,False,False,True,False,True,True,True,False,False,False
2,False,True,False,False,True,False,False,False,False,False,...,False,False,True,False,True,True,True,False,False,False
3,False,True,False,False,True,True,True,False,False,False,...,False,False,True,False,True,True,True,False,False,False
4,False,True,False,False,True,True,True,False,False,False,...,False,False,True,False,True,True,True,False,False,False


In [426]:
odds_df.head()

,barstool_1_odds,betclic_1_odds,betfair_1_odds,betfred_1_odds,betmgm_1_odds,betonlineag_1_odds,betrivers_1_odds,betus_1_odds,betway_1_odds,bovada_1_odds,...,sugarhouse_1_odds,superbook_1_odds,twinspires_1_odds,unibet_1_odds,unibet_eu_1_odds,unibet_uk_1_odds,unibet_us_1_odds,williamhill_1_odds,williamhill_us_1_odds,wynnbet_1_odds
0,1.10,1.10,0.0,0.0,1.13,0.00,0.00,0.00,0.0,1.09,...,0.0,0.00,1.13,0.0,1.13,1.12,1.14,0.00,0.00,0.00
1,1.05,1.06,0.0,0.0,1.05,0.00,0.00,0.00,0.0,1.05,...,0.0,0.00,1.01,0.0,1.01,1.04,1.04,0.00,0.00,0.00
2,1.83,1.85,0.0,0.0,1.87,0.00,0.00,0.00,0.0,1.77,...,0.0,0.00,1.82,0.0,1.82,1.89,1.85,0.00,0.00,0.00
3,2.10,2.15,0.0,0.0,2.10,2.14,2.16,2.16,0.0,2.08,...,0.0,2.15,2.15,0.0,2.16,2.15,2.16,2.10,2.10,2.15
4,1.77,1.78,0.0,0.0,1.77,1.79,1.74,1.77,0.0,1.76,...,0.0,1.78,1.74,0.0,1.74,1.74,1.74,1.77,1.77,1.80


In [427]:
odds_df_masked = odds_df.where(mask, 0)

In [428]:
odds_df_masked.head()

,barstool_1_odds,betclic_1_odds,betfair_1_odds,betfred_1_odds,betmgm_1_odds,betonlineag_1_odds,betrivers_1_odds,betus_1_odds,betway_1_odds,bovada_1_odds,...,sugarhouse_1_odds,superbook_1_odds,twinspires_1_odds,unibet_1_odds,unibet_eu_1_odds,unibet_uk_1_odds,unibet_us_1_odds,williamhill_1_odds,williamhill_us_1_odds,wynnbet_1_odds
0,0.0,1.10,0.0,0.0,1.13,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,1.13,0.0,1.13,1.12,1.14,0.0,0.0,0.0
1,0.0,1.06,0.0,0.0,1.05,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,1.01,0.0,1.01,1.04,1.04,0.0,0.0,0.0
2,0.0,1.85,0.0,0.0,1.87,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,1.82,0.0,1.82,1.89,1.85,0.0,0.0,0.0
3,0.0,2.15,0.0,0.0,2.10,2.14,2.16,0.0,0.0,0.0,...,0.0,0.0,2.15,0.0,2.16,2.15,2.16,0.0,0.0,0.0
4,0.0,1.78,0.0,0.0,1.77,1.79,1.74,0.0,0.0,0.0,...,0.0,0.0,1.74,0.0,1.74,1.74,1.74,0.0,0.0,0.0


In [429]:
subset_columns

['barstool_1_odds',
 'betmgm_1_odds',
 'bovada_1_odds',
 'coral_1_odds',
 'draftkings_1_odds',
 'fanduel_1_odds']

In [430]:
odds_df_masked[subset_columns].head()

,barstool_1_odds,betmgm_1_odds,bovada_1_odds,coral_1_odds,draftkings_1_odds,fanduel_1_odds
0,0.0,1.13,0.0,0.0,1.11,1.08
1,0.0,1.05,0.0,0.0,1.03,1.05
2,0.0,1.87,0.0,0.0,1.80,1.77
3,0.0,2.10,0.0,0.0,2.10,2.16
4,0.0,1.77,0.0,0.0,1.77,1.74


In [431]:
df['highest_bettable_odds'] = odds_df_masked[subset_columns].max(axis=1)

In [432]:
df['highest_bettable_odds'].head()

0    1.13
1    1.05
2    1.87
3    2.16
4    1.77
Name: highest_bettable_odds, dtype: float64

In [444]:
df = pd.read_parquet('/Users/stefanfeiler/Desktop/Smartbetter_site/website/mlb_data/2023_data_for_val.parquet')

In [446]:
df.tail(1)

,team_1,team_2,game_id,commence_time,time_pulled,home_team,away_team,barstool_1_odds,barstool_1_time,barstool_2_odds,...,my_game_id_final,number_of_game_today,day_of_week,away_team_league,away_team_game_number,home_team_league,home_team_game_number,day_night,park_id,winning_team
282371,Tampa Bay Rays,Texas Rangers,885c435a882bb2570a2b197844bf15bc,2023-07-17 17:05:00,2023-07-16 10:55:00,Texas Rangers,Tampa Bay Rays,1.87,2023-07-16 10:55:00,1.95,...,Tampa Bay RaysTexas Rangers20230717_0,0,Mon,AL,97.0,AL,95.0,N,ARL03,Texas Rangers


150